<a href="https://colab.research.google.com/github/deenyse/VSB_ZSU/blob/main/Copy_of_UASS_6_18_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analýza dynamických sítí

> Cílem cvičení je vyzkoušet si analyzovat dynamické sítě s využitím poznatků z minulých cvičení.


## Část 1: Struktury v sítí

1. Sítě
  - Emailová síť [Emailova sit](https://homel.vsb.cz/~pap0081/email_network.zip)
  - Vyberte si jednu síť síť https://homel.vsb.cz/~kud007/lectures/uass_04.pdf

2. Rozdělte síť alespoň na 5 statických snímků.

3. Analyzujte síť, výstupem by měl být dokument, kde shrnete získané poznatky v rozdílech mezi snímky
  - Zjistěte, jak se v čase (v jednotlivých snímcích) mění průměrný stupeň a průměrný vážený stupeň sítě.
  - Zjistěte, jak se v čase mění počet komunit a souvislých komponent, a jak se mění průměrná a maximální velikost komunity.
  - Vizualizujte jednotlivé sítě a popřemýšlejte, jaké typické vzorce se v sítích vyskytují a v jakém množství (kliky a hvězdy, malé souvislé komponenty a outliers apod.).
  - Ve vizualizaci najděte nějaké vrcholy s vysokým stupněm resp. s vysokým váženým stupněm a popřemýšlejte, proč se v některých případech nejedná o tytéž vrcholy a zkuste zformulovat, proč tomu tak je.
  - V alespoň jedné síti vyberte alespoň jeden vrchol, který má na posledním snímku vysoký stupeň resp. vážený stupeň a na něm ukažte, jak se vyvíjí čase (postupné změny ve stupni resp. váženém stupni, velikost komunity popř. komponenty, do které patří).

In [ ]:
# Nacteni site

import networkx as nx
from datetime import datetime

G = nx.Graph()
delimiter = ','

with open("email-dnc.edges", "r") as f:
    for line in f:
        u, v, ts = line.strip().split(delimiter)
        ts = float(ts)
        dt = datetime.fromtimestamp(ts)
        G.add_edge(u, v, timestamp=dt)



In [ ]:
print(G)

Graph with 6809 nodes and 7697 edges


In [ ]:
import random
import networkx as nx

def find_components(G):
    return list(nx.connected_components(G))


def find_cliques(G, min_size=3):
    clique_nodes = set()
    for clique in nx.find_cliques(G):
        if len(clique) >= min_size:
            clique_nodes.update(clique)
    return clique_nodes


def find_stars(G, degree_percentile=80, leaf_density_threshold=0.2):
    degrees = [G.degree(n) for n in G]
    min_degree = np.percentile(degrees, degree_percentile)
    star_nodes = set()
    for node in G:
        if G.degree(node) < min_degree:
            continue
        neighbors = list(G.neighbors(node))
        if len(neighbors) < 3:
            continue
        if nx.density(G.subgraph(neighbors)) <= leaf_density_threshold:
            star_nodes.add(node)
    return star_nodes



In [ ]:
def color_nodes(G, nodes, color, node_type=None):
    for n in nodes:
        if n not in G.nodes:
            continue
        G.nodes[n]["color"] = color
        if node_type:
            G.nodes[n]["type"] = node_type


def export_to_gephi(G: nx.Graph, path: str = "graph_colored.gexf"):
    nx.write_gexf(G, path)



In [ ]:

    # Detect structures
    clique_nodes = find_cliques(G, min_size=4)
    star_nodes = find_stars(G, degree_percentile=80, leaf_density_threshold=0.2)
    components = list(find_components(G))

    # Initialize default color/type
    nx.set_node_attributes(G, "default", "type")
    nx.set_node_attributes(G, "gray", "color")

    # Color by structure type
    color_nodes(G, clique_nodes, "#ff0000", "clique")
    color_nodes(G, near_nodes, "#ff9900", "near_clique")
    color_nodes(G, star_nodes, "#00ccff", "star")

    # Print summary
    print(f"Total nodes: {G.number_of_nodes()}")
    print(f"Components: {len(components)}")
    print(f"Clique nodes: {len(clique_nodes)}")
    print(f"Near-clique nodes: {len(near_nodes)}")
    print(f"Star nodes: {len(star_nodes)}")



In [ ]:
import networkx as nx
from collections import defaultdict
from datetime import datetime, timedelta

def split_graph_into_snapshots_datetime(G: nx.Graph, time_attr: str, num_snapshots: int,
                                        start_time=None, end_time=None, attr_type='edge'):
    times = []
    if attr_type == 'edge':
        for _, _, data in G.edges(data=True):
            if time_attr in data:
                times.append(data[time_attr])
    elif attr_type == 'node':
        for _, data in G.nodes(data=True):
            if time_attr in data:
                times.append(data[time_attr])
    else:
        raise ValueError("attr_type must be 'edge' or 'node'")

    if not times:
        raise ValueError("No time attributes found in the graph.")

    start = start_time if start_time else min(times)
    end = end_time if end_time else max(times)
    print(start, end)
    total_seconds = (end - start).total_seconds()
    interval_seconds = total_seconds / num_snapshots

    for i in range(num_snapshots):
      print(datetime.fromtimestamp(start.timestamp() + interval_seconds * i))

    def get_snapshot_index(t):
        delta_sec = (t - start).total_seconds()
        idx = int(delta_sec / interval_seconds)
        return max(0, min(idx, num_snapshots - 1))

    graph_type = nx.DiGraph if G.is_directed() else nx.Graph
    snapshots = defaultdict(graph_type)

    if attr_type == 'edge':
        for u, v, data in G.edges(data=True):
            if time_attr in data:
                idx = get_snapshot_index(data[time_attr])
                snapshots[idx].add_edge(u, v, **data)

        for idx, H in snapshots.items():
            used_nodes = set(H.nodes())
            for n in used_nodes:
                if n in G.nodes:
                    H.nodes[n].update(G.nodes[n])

            iso = list(nx.isolates(H))
            H.remove_nodes_from(iso)

    elif attr_type == 'node':
        for node, data in G.nodes(data=True):
            if time_attr in data:
                idx = get_snapshot_index(data[time_attr])
                snapshots[idx].add_node(node, **data)

        for idx, H in snapshots.items():
            for u, v, data in G.edges(data=True):
                if u in H and v in H:
                    H.add_edge(u, v, **data)

            iso = list(nx.isolates(H))
            H.remove_nodes_from(iso)

    return dict(snapshots)

In [ ]:
for k,v in split_graph_into_snapshots_datetime(G, 'timestamp', 5).items():
  print(v)

2013-09-16 02:30:33 2016-05-25 06:07:30
2013-09-16 02:30:33
2014-03-31 12:49:56.400000
2014-10-13 23:09:19.800000
2015-04-28 09:28:43.200000
2015-11-10 19:48:06.600000
Graph with 1790 nodes and 4308 edges
Graph with 133 nodes and 137 edges
Graph with 24 nodes and 20 edges
Graph with 2 nodes and 1 edges
